In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

<a id="notebook_id"></a>
# Interfaces: Linked list-based stack

The `ArrayList`-based and array-based stack implementations have `push` operations that are in $O(1)$ amortized complexity; however, when the internal array holding the elements
runs out of capacity, the array must be reallocated and its elements copied into the new array which leads to a worst-case $O(n)$ complexity. It is possible to achieve $O(1)$ worst-case
complexity for `push`, but we cannot use an array to store the elements of the stack.

Consider a stack having exactly one element: The string `"A"`. We can draw a picture of such a stack like so:

![A stack with one element](../resources/images/stacks/linked-list-1.png)

The box and arrow represents an object called a *node*. A node has a reference to an element (the string `"A"` in this case) and a reference to the next element in the sequence
(called a *link* and represented by the arrow).
Because the stack has only one element, there is no node for the next element; this is indicated by the arrow pointing to `null` (remember that in Java, `null` is the reference to no object).
The node is shown in blue to indicate that it holds the top element of the stack.

Suppose that we push the string `"B"` onto the top of the stack. Our stack now looks like so:

![A stack with two elements](../resources/images/stacks/linked-list-2.png)

The node holding the `"B"` is now labelled `top` and its arrow points to the node holding the `"A"`.

Suppose that we push the string `"C"` onto the top of the stack. Our stack now looks like so:

![A stack with three elements](../resources/images/stacks/linked-list-3.png)

The node holding the `"C"` is now labelled `top` and its arrow points to the node holding the `"B"`.

Notice that the nodes are linked in a linear structure. It is possible to reach any node in the sequence by starting from the `top` node and following the link to the next node in the
sequence.

The data structure shown above is called a *singly linked list* because it represents a sequence of elements where each element is joined by one link.

<div class="alert alert-info">
    Java's <tt>LinkedList</tt> is a doubly linked list. Each node has two links, one to the next node in the sequence and second to the previous node in the sequence.
</div>

Pushing an element onto to the top of the stack always requires a constant number of elementary operations:

1. make a new node to hold the element
2. set the link of the new node to point to the current `top` node
3. set `top` to refer to the new node

Thus, the `push` operation has worst-case complexity $O(1)$ for a linked list-based stack.

The remainder of this notebook illustrates how to implement a linked list-based stack class named `LinkedStack`.

## Fields

Our stack class will have one field that keeps count of the number of elements currently on the stack (the size of the stack).

A second field will be a reference to the top node.

Our class with its two fields will be something like:

```java
public class LinkedStack {
    
    // the number of elements currently on the stack
    private int size;
    
    // the node containing the top element of the stack
    private Node top;
    
}
```

### Static nested classes

The `Node` class will look something like:

```java
static class Node {
    
    // the element stored in the node
    String elem;
    
    // the link to the next node in the sequence
    Node next;
    
    Node(String elem, Node next) {
        this.elem = elem;
        this.next = next;
    }
}
```

The user of our stack class does not care about nodes; the user wants to push and pop elements. Furthermore, the `Node` class is not especially useful outside of our
stack class. We can hide the `Node` class from other classes by defining the `Node` class inside of the `LinkedStack` class and setting the access modifier to `private`:

In [ ]:
public class LinkedStack {
    
    // the number of elements currently on the stack
    private int size;
    
    // the node containing the top element of the stack
    private Node top;
    
    // static nested class representing nodes of the linked list
    private static class Node {
            
        // the element stored in the node
        String elem;
    
        // the link to the next node in the sequence
        Node next;
    
        Node(String elem, Node next) {
            this.elem = elem;
            this.next = next;
        }
    }
    
}

In Java a *nested class* is a class defined inside of another class (called its *enclosing class*). A nested class should exist only to serve its enclosing class; if a nested class
would be useful on its own then it should be top-level class.

A nested class is a member of its enclosing class which means that the enclosing class has access to all of the members of the nested class including `private` members. In our case,
the `LinkedStack` class has access to the `Node` fields `elem` and `next`, and the `Node` constructor.

A *static member class* is the simplest kind of nested class in Java and it is the kind of class that is used for our `Node` class. A static member class is essentially an ordinary class that
happens to be declared inside of another class. An instance of a static member class can exist in isolation from an instance of the enclosing class. As with static methods and static fields, a
static nested class is associated with its outer class (instead of being associated with objects of the outer class). A static nested class cannot refer directly to fields or non-static methods
defined in its enclosing class. In our case, the `Node` class does not have access to the `size` and `top` fields of its enclosing class; this causes no problems in our stack
implementation because an individual node never needs access to this information.

<div class="alert alert-info">
    The other three kinds of nested class in Java are different kinds of <i>inner classes</i>. Inner classes are not <tt>static</tt> classes.
    An inner class requires instances of such classes to always have an associated instance of the enclosing class; the compiler adds a hidden reference to the enclosing
    class to every instance of a non-static nested class. This hidden reference costs memory to store the reference which is why a nested class should be declared static if possible.
    Interested readers can refer to the <it><a href="https://docs.oracle.com/javase/tutorial/java/javaOO/nested.html">Nested class tutorial</a></it> for additional details.
</div>

Notice that the `Node` class has no methods. The `Node` class is an example of a class whose sole purpose is to gather information into a single object; in other words,
a `Node` object represents purely data. The user of the class
(in this case the user is the `LinkedStack` class) accesses the information stored in a `Node` by directly accessing the fields. Such data structures (called *records* or
*structures*) are built in to many other programming languages.

<div class="alert alert-info">
    Java 14 introduced Java records to the language as a preview feature, and records were officially added to the language in Java 16. Interested readers
    can read more about <a href="https://docs.oracle.com/en/java/javase/16/language/records.html">records here</a>.
</div>

## Constructor

The constructor initializes an empty stack by setting `this.size` to `0` and `this.top` to `null` (indicating that there is no top `Node` object).

## `size` method

The `size` method simply returns `this.size`.

## `push(String elem)` method

To push an element onto the stack we:

1. make a new node to hold the element
2. set the link of the new node to point to the current top node
3. set `this.top` to refer to the new node
4. increment `this.size`
    * because we have added one element to the top of the stack

## `pop()` method

To pop an existing element we:

1. obtain a reference to the element stored in `this.top`,
2. set `this.top` to `this.top.next`
    * this makes the next element in the stack be the top of the stack
3. decrement `this.size`
    * because we have removed one element from the stack
4. return the reference obtained in Step 1 to return the popped element

## Java implementation

The class `ListStack` shown below implements the stack described above. Run the next cell to compile the class:

In [ ]:
%classpath add jar ../resources/jar/notes.jar

import ca.queensu.cs.cisc124.notes.interfaces.Stack;

public class LinkedStack implements Stack {
    // the number of elements currently on the stack
    private int size;
    
    // the node containing the top element of the stack
    private Node top;
    
    // static nested class representing nodes of the linked list
    private static class Node {
            
        // the element stored in the node
        String elem;
    
        // the link to the next node in the sequence
        Node next;
    
        Node(String elem, Node next) {
            this.elem = elem;
            this.next = next;
        }
    }
    
    
    public LinkedStack() {
        this.size = 0;
        this.top = null;
    }


    @Override
    public int size() {
        return this.size;
    }


    @Override
    public void push(String elem) {
        Node n = new Node(elem, this.top);
        this.top = n;
        this.size++;
    }


    @Override
    public String pop() {
        if (this.isEmpty()) {
            throw new RuntimeException("popped an empty stack");
        }
        String popped = this.top.elem;
        this.top = this.top.next;
        this.size--;
        return popped;
    }
    
    
    @Override
    public String toString() {
        StringBuilder b = new StringBuilder("Stack:");
        Node n = this.top;
        while (n != null) {
            b.append('\n');
            b.append(n.elem);
            n = n.next;
        }
        return b.toString();
    }
}


The following cell contains a small program that uses the `ListStack` class:

In [ ]:
%classpath add jar ../resources/jar/notes.jar
import ca.queensu.cs.cisc124.notes.interfaces.Stack;

Stack t = new LinkedStack();
t.push("A");
t.push("B");
t.push("C");
System.out.println("size: " + t.size());
System.out.println(t);
System.out.println();

String popped = t.pop();
System.out.println("popped: " + popped);
System.out.println("size: " + t.size());
System.out.println(t);
System.out.println();

popped = t.pop();
System.out.println("popped: " + popped);
System.out.println("size: " + t.size());
System.out.println(t);
System.out.println();

popped = t.pop();
System.out.println("popped: " + popped);
System.out.println("size: " + t.size());
System.out.println(t);
System.out.println();

// force an exception
t.pop();

### Iterating over a linked list

The `toString` method illustrates how to iterate over a linked list. We start by setting a `Node` reference `n` to `this.top`. We then use a `while` loop that runs as long as `n` is
not a `null` reference. In the body of the `while` loop, the last statement sets `n` to refer to the next node in the sequence. The general pattern for iterating over a linked list
starting at the first node of the sequence is:

```java
Node n = first;
while (n != null) {
    // do something with the node here
    
    n = n.next;    // make n refer to the next node in the sequence
}
```

## Exercises

1. What is the main difference between a static nested class and an inner class?

2. Add a method `boolean contains(String s)` to the `LinkedStack` class that returns `true` if the stack contains at least one element equal to `s`.

3. Add a method `String bottom()` to the `LinkedStack` class that returns the element at the bottom of the stack. Do not add any fields to the class. Throw an
exception if the stack is empty (because there is no bottom element in an empty stack).

4. [Source inspiration](https://leetcode.com/problems/min-stack/). Implement a linked list-based stack class called `MinStack` that provides `push`, `pop`, and `shortest` operations
having $O(1)$ complexity. The `shortest` operation should return the shortest string in the stack.
    * *Hint*: Add a new field to the `Node` class that stores a reference to the shortest string stored in this node or in any node after this node.